## From biological to artifical neurons

Origin: McCulloch and Pitts' 1943 paper "A Logical Calculus of Ideas Immanent in Nervous Activity". This paper presented a simplified computational model of how biological neurons might work together in animal brains to perform complex computations using propotional logic.

Long winter followed, till the 1980s. In the early 1980s, new architectures were invented and better training techniques were developed.

Currently, there's another wave of interest growing on ANNs. Why not stop this time?
- Huge amounts of data, and neural networks outperform other ML models on very large and complex problems

- Increase in computing power

- Good results lead to investment, and investment leads to good results!